# NIR 2022 - Lab 4: LMs & Query Rewriting

Today, we will see how to run approaches based on language models and query rewriting in PyTerrier.

## Systems Setup

We will start by building an index of our data collection and a few systems in PyTerrier.
This step is only required to obtain system outputs.

In [7]:
# Load the data
import pandas as pd

# corpus
docs_df = pd.read_csv('data/lab_docs.csv', dtype=str)
print(docs_df.shape)
print(docs_df.head())

# topics
topics_df = pd.read_csv('data/lab_topics.csv', dtype=str)
print(topics_df.shape)
print(topics_df.head())

# Load qrels
qrels_df = pd.read_csv('data/lab_qrels.csv', dtype=str)
print(qrels_df.shape)
print(qrels_df.head())

(2453, 2)
     docno                                               text
0   935016  he emigrated to france with his family in 1956...
1  2360440  after being ambushed by the germans in novembe...
2   347765  she was the second ship named for captain alex...
3  1969335  world war ii was a global war that was under w...
4  1576938  the ship was ordered on 2 april 1942 laid down...
(9, 2)
       qid                 query
0  1015979    president of chile
1     2674    computer animation
2   340095  2020 summer olympics
3  1502917         train station
4     2574       chinese cuisine
(2454, 4)
       qid    docno label iteration
0  1015979  1015979     2         0
1  1015979  2226456     1         0
2  1015979  1514612     1         0
3  1015979  1119171     1         0
4  1015979  1053174     1         0


In [8]:
# Init PyTerrier
import pyterrier as pt
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])  # Initialisation package for RM3

In [9]:
# Build index
indexer = pt.DFIndexer("./indexes/default", overwrite=True, blocks=True)
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Number of documents: 2453
Number of terms: 23693
Number of postings: 208487
Number of fields: 0
Number of tokens: 273373
Field names: []
Positions:   true



In [10]:
# Load index
index_ref = pt.IndexRef.of("./indexes/default/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Number of documents: 2453
Number of terms: 23693
Number of postings: 208487
Number of fields: 0
Number of tokens: 273373
Field names: []
Positions:   true



In [11]:
# Build IR systems
tf = pt.BatchRetrieve(index, wmodel="Tf")
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [12]:
# Evaluate systems on the first three topics using the PyTerrier Experiment interface
qrels_df = qrels_df.astype({'label': 'int32'})
pt.Experiment(
    retr_systems=[tf, tfidf, bm25,],
    names=['TF', 'TF-IDF', 'BM25'],
    topics=topics_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"])

,name,map,ndcg,ndcg_cut_10,P_10
0,TF,0.610184,0.789583,0.851008,0.800000
1,TF-IDF,0.622287,0.798228,0.840808,0.766667
2,BM25,0.628454,0.800955,0.842503,0.766667


## Language Models

Language models (LMs) are an alternative view to the ranking problem.
In LMs, the actual occurrences of terms in a document are compared with the expected occurrences predicted from the characteristics of the collection and the document.

Given a document $d$, that is assumed to be relevant, we develop a LM to estimate $p(q|d)$, the probability that query $q$ would be entered to retrieve document $d$. Documents are then ranked according to these probabilities.

In practice, given a document $d$, the goal is to construct a document LM for the queries that may be entered to retrieve it.
The simplest document LM is based on the counts of the terms appearing in the document (maskimum likelihood model).
However, given the limited number of terms within a document, the resulting estimates (i) might be wildly inaccurate, and (ii) equal to $0$ for all the terms not appearing in the document.
To address these problems, a common approach in information retrieval is to use _smoothing_ techniques.

In this notebook, we will use PyTerrier's implementation of the [Dirichlet LM](http://terrier.org/docs/v4.0/javadoc/org/terrier/matching/models/DirichletLM.html), a language model with Dirichlet smoothing, a technique that pretends that each document has an extra $\mu>0$ tokens in each document.
As a result, the impact of additional terms depends on the length of a given document: the longer the document, the lower the impact.

In [13]:
# Dirichlet Language Model
dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")

In PyTerrier, the deafult value of $\mu$ is $\mu=2500$.
During the class, however, we have seen that a common approach is to set $\mu$ to the average document length in the collection.
Compute the average document length $ADL$ and evaluate a Dirichlet LM with $\mu=ADL$.

Tips: 
- Remember that you can access a `document index` within a PyTerrier `index`. 
- Also, PyTerrier assign an incremental document id to each document in the collection, starting from $0$.

In [14]:
# Dirichlet Language Model with parameter mu equal to average document length

avg = 10

dlm_avg = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"c": avg})

In [15]:
# Evaluate systems on the topics using the PyTerrier Experiment interface
pt.Experiment(
    retr_systems=[tf, tfidf, bm25, dlm, dlm_avg],
    names=['TF', 'TF-IDF', 'BM25', 'DLM_default', 'DLM_avg'],
    topics=topics_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"])

,name,map,ndcg,ndcg_cut_10,P_10
0,TF,0.610184,0.789583,0.851008,0.800000
1,TF-IDF,0.622287,0.798228,0.840808,0.766667
2,BM25,0.628454,0.800955,0.842503,0.766667
3,DLM_default,0.644624,0.812032,0.865656,0.788889
4,DLM_avg,0.644624,0.812032,0.865656,0.788889


## Pipelines

Before looking at query rewriting techniques, we will have a look at PyTerrier pipelines, which will be used in the rest of this notebook.

Part of the power of PyTerrier comes indeed from having these pipelines, that allow to easily formulate complex retrieval pipelines.
Particularly relevant to us, today, is the chaining pipeline, which is obtained with the `>>` (Then) operator.
This operator allows us to perform multiple steps one after the other.

For example, as we will see later in the course, a typical approach consists of re-ranking documents that were first retrieved by another system.
Given a query $Q$ and two ranking systems $\text{R}_1$ and $\text{R}_2$, we can perform the following pipeline:
$$ R' = \text{R}_2(\text{R}_1(Q)))$$
through the `>>` operator:
$$\text{Pipeline} = R_1 >> R_2$$
$$R' = \text{Pipeline}(Q)$$
where $R'$ is the list of ranked documents.

In the following, we will be adding query expansion as an intermediate step between re-ranking in order to perform ranking with pseudo-relevance feedback.

## Query Rewriting

Query rewriting consists of reformulating the original query in order to improve the effectiveness of a ranking system. 
The basic idea behind this technique is to return relevant documents even if there is no term that matches with the original query, hence possibly increasing the quality of the ranking.

Although queries can be rewritten by users, in this notebook, we will look at automatic techniques to query rewriting.
In particular, PyTerrier differentiates between two forms of query rewriting:

- Query expansion `Q -> Q`: this rewrites a query $Q$, for instance by adding/removing extra query terms. 

- Pseudo-relevance feedback `R -> Q`: this rewrites a query $Q$ by making use of an associated set of documents $R$.


### Query Expansion

PyTerrier offers native access to Metzler and Croft’s sequential dependence model (`sdm`), designed to boost the scores of documents where the query terms occur in close proximity.
This technique rewrites each input query such that:
- pairs of adjacent query terms are added as #1 and #uw8 complex query terms, with a low weight
- the full query is added as #uw12 complex query term, with a low weight.
- all terms are weighted by a proximity model, such as the Dirichlet LM

For example, the query "pyterrier IR platform" would become "pyterrier IR platform #1(pyterrier IR) #1(IR platform) #uw8(pyterrier IR) #uw8(IR platform) #uw12(pyterrier IR platform)".

In [ ]:
# First, expand queries through the sequential dependence model
# Then, use BM25 to rank the documents in the collection using the expanded queries
sdm = pt.rewrite.SequentialDependence()
sdm_bm25 = sdm >> bm25

We can look at how the query ""black wall" is expanded by SDM as follows:

In [ ]:
sdm_bm25.search("black wall").iloc[0]['query']

In [ ]:
# Evaluate systems on the topics using the PyTerrier Experiment interface
pt.Experiment(
    retr_systems=[tf, tfidf, bm25, dlm, dlm_avg, sdm_bm25],
    names=['TF', 'TF-IDF', 'BM25', 'DLM_default', 'DLM_avg', 'SDM >> BM25'],
    topics=topics_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"])

#### Query Expansion with word vectors

Another popular approach to query expansion is to use word vectors, such as Word2Vec or Glove, to retrieve similar words to the ones in the original query.
While we will cover word vectors in the upcoming classes, we see here how we can use them to perform query expansion.

In particular, we use the Gensim library to download 50-dimensional Glove word vectors and retrieve $k$ similar words for each term in the query.

In [ ]:
!pip install --upgrade gensim

In [ ]:
import gensim
import gensim.downloader as api

In [ ]:
# Download and load Glove vectors
model = api.load('glove-wiki-gigaword-50')
print(model.most_similar('tree'))

In [ ]:
# Expand each query by adding the top-k similar words for each word in the query
k = 10

topics_qe_df = topics_df.copy()
for i in range(len(topics_qe_df)):
    q = topics_qe_df.iloc[i]['query']
    qe = []
    for word in q.split(' '):
        expanded_words = [pair[0] for pair in model.most_similar(word, topn=k)]
        expanded_words.append(word)
        qe.append(expanded_words)
    topics_qe_df.iloc[i]['query'] = gensim.parsing.preprocessing.remove_stopwords(" ".join([e for l in qe for e in l]))

topics_qe_df

In [ ]:
# Original query
topics_df.iloc[1]['query']

In [ ]:
# Expanded query
topics_qe_df.iloc[1]['query']

In [ ]:
# Evaluate previous systems on the expanded topics
pt.Experiment(
    retr_systems=[tf, tfidf, bm25, dlm, dlm_avg],
    names=['TF', 'TF-IDF', 'BM25', 'DLM_default', 'DLM_avg'],
    topics=topics_qe_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"])

### Pseudo-relevance Feedback

Finally, we now look at pseudo-relevance feedback approaches.

Pseudo-relevance feedback allows to re-rank documents without requiring the user to select relevant documents among the ones retrieved.
It does so by assume that the top-$k$ ranked documents are relevant, and then applies relevance feedback under this assumption.

PyTerrier offers native access to several approaches to pseudo-relevance feedback.
Here, we will look at RM3, one of the most popular and effective approaches.

The following parameters can be used to tune the behavioor of the model:
- `fb_terms`: number of feedback terms (Defaults to 10)
- `fb_docs`: number of feedback documents (Defaults to 3)
- `fb_lambda`: lambda in RM3, i.e. importance of relevance model viz feedback model (Defaults to 0.6)

In [ ]:
# Rank by BM25, then apply query rewriting with RM3, finally re-rank the documents with BM25 using rewritten queries
rm3 = pt.rewrite.RM3(index)
rm3_pipe = bm25 >> rm3 >> bm25

In [ ]:
# Evaluate systems
pt.Experiment(
    retr_systems=[tf, tfidf, bm25, dlm, dlm_avg, sdm_bm25, rm3_pipe],
    names=['TF', 'TF-IDF', 'BM25', 'DLM_default', 'DLM_avg', 'SDM >> BM25', 'BM25 >> RM3 >> BM25'],
    topics=topics_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"]
)

## Fusion Techniques

As a last, quick task in this lab, we will look at a rank fusion method.

Rank fusion refers to the technique that merges multiple system runs to produce a single top-$k$ list.
Widely used approaches are:
- CombSUM (score-based)
- Borda Count (rank-based)
- Reciprocal Rank Fusion (rank-based)

We now look at CombSUM, which combines multiple system runs with a (weighted) sum of their scores.
This can be trivially implemented in PyTerrier with the `+` operator:

In [ ]:
bm25_plus_tfidf = 1.0*bm25 + 1.0*tfidf

In [ ]:
# Evaluate systems
pt.Experiment(
    retr_systems=[tf, tfidf, bm25, dlm, dlm_avg, sdm_bm25, rm3_pipe, bm25_plus_tfidf],
    names=['TF', 'TF-IDF', 'BM25', 'DLM_default', 'DLM_avg', 'SDM >> BM25', 'BM25 >> RM3 >> BM25', 'CombSUM'],
    topics=topics_df,
    qrels=qrels_df,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "P_10"]
)